#  implementation of LSTM for experiment

In [56]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [62]:
learning_rate = 0.001
training_step = 10000
batch_size = 128
display_step = 200


tf.reset_default_graph()

## network architecture features 
num_input = 28
timestamp = 28
num_hidden = 128
num_class =10

X = tf.placeholder("float",[None,timestamp,num_input])
Y = tf.placeholder("float",[None,num_class])


In [63]:
# define the weights matrix for  Single forward RNN

# weights = {
#     "out": tf.Variable(tf.random_normal([num_hidden,num_class]))
# }

# bias = {
#     "out": tf.Variable(tf.random_normal([num_class]))
# }




## defining the weights for bi directional RNN
weights = {
    "out": tf.Variable(tf.random_normal([2*num_hidden,num_class]))
}

bias = {
    "out": tf.Variable(tf.random_normal([num_class]))
}


In [64]:
def RNN(x,weight,bias):
    ## putting the x into the data
    x = tf.unstack(x,timestamp,1)
    
    lstm_cell = rnn.BasicLSTMCell(num_hidden,forget_bias = 1.0)
    
    ## getting th value of lstm cell 
    output, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(output[-1], weights['out']) + bias['out']


In [65]:
def bidirectionalRNN(x,weight,bias):
    ## using bidirectional RNN for computation
    ## putting the x into the data
    x = tf.unstack(x,timestamp,1)
    
    lstm_fwrd_cell = rnn.BasicLSTMCell(num_hidden,forget_bias = 1.0)
    
    lstm_bckwd_cell = rnn.BasicLSTMCell(num_hidden,forget_bias = 1.0)
    ## getting th value of lstm cell 
    output, _, _ = rnn.static_bidirectional_rnn(lstm_fwrd_cell, lstm_bckwd_cell, x,
                                              dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(output[-1], weights['out']) + bias['out']


In [66]:
logits = bidirectionalRNN(X,weights,bias)
prediction = tf.nn.softmax(logits)
## NOTE: ON getting VALUE error run from start 

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [67]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1,training_step,1):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        ## reshape the data to 28 x 28
        batch_x = batch_x.reshape((batch_size,timestamp,num_input))
         # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timestamp, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 3.2825, Training Accuracy= 0.125
Step 200, Minibatch Loss= 2.1779, Training Accuracy= 0.211
Step 400, Minibatch Loss= 2.0355, Training Accuracy= 0.367
Step 600, Minibatch Loss= 1.8564, Training Accuracy= 0.398
Step 800, Minibatch Loss= 1.6660, Training Accuracy= 0.500
Step 1000, Minibatch Loss= 1.6533, Training Accuracy= 0.516
Step 1200, Minibatch Loss= 1.5927, Training Accuracy= 0.516
Step 1400, Minibatch Loss= 1.5926, Training Accuracy= 0.516
Step 1600, Minibatch Loss= 1.3344, Training Accuracy= 0.594
Step 1800, Minibatch Loss= 1.3505, Training Accuracy= 0.609
Step 2000, Minibatch Loss= 1.2230, Training Accuracy= 0.656
Step 2200, Minibatch Loss= 1.2605, Training Accuracy= 0.586
Step 2400, Minibatch Loss= 1.2378, Training Accuracy= 0.617
Step 2600, Minibatch Loss= 1.3130, Training Accuracy= 0.555
Step 2800, Minibatch Loss= 1.2408, Training Accuracy= 0.641
Step 3000, Minibatch Loss= 1.1846, Training Accuracy= 0.641
Step 3200, Minibatch Loss= 0.9529, Training Acc